# Exploring LittlePay's Authorisations Data

In [1]:
from calitp import get_engine, query_sql, magics

In [2]:
import os
os.environ['AIRFLOW_ENV'] = 'development'

## Initial exploration

* How big is the table?
* What are the fields available?

In [3]:
%%sql

select count(*) from payments.stg_enriched_authorisations

,f0_
0,55949


In [4]:
%%sql

select *
from payments.stg_enriched_authorisations
limit 10

,participant_id,aggregation_id,acquirer_id,request_type,transaction_amount,currency_code,retrieval_reference_number,response_code,status,authorisation_date_time_utc,calitp_extracted_at,calitp_hash,calitp_file_name,calitp_export_account,calitp_export_datetime,calitp_n_dupes,calitp_n_dupe_ids,calitp_dupe_number
0,sbmtd,e56cdf95-d371-403a-9f74-306a6fe90a71,CS,CARD_CHECK,0.0,840,None,None,UNAVAILABLE,2021-09-29 01:46:38+00:00,2022-02-22,Wh69ea2YsKVmvq6sg9oXZw==,gs://gtfs-data-test/payments-processed/authori...,sbmtd,2021-09-30 04:44:00,1,1,1
1,mst,3854286c-19b3-48a3-9d30-60e28bdc13de,CS,CARD_CHECK,0.0,840,6207816454776574504053,100,VERIFIED,2021-05-12 01:07:26+00:00,2022-02-22,W9Qp2/mBFdAKtg2kH6Plcw==,gs://gtfs-data-test/payments-processed/authori...,mst,2021-06-18 02:24:00,2,2,2
2,mst,3854286c-19b3-48a3-9d30-60e28bdc13de,CS,CARD_CHECK,0.0,840,6207816454776574504053,100,VERIFIED,2021-05-12 01:07:26+00:00,2022-02-22,W9Qp2/mBFdAKtg2kH6Plcw==,gs://gtfs-data-test/payments-processed/authori...,mst,2021-06-18 02:24:00,2,2,1
3,mst,9335c6ca-fd5a-41ac-9b35-c6d97eb2b14c,CS,CARD_CHECK,0.0,840,6207937648856229104091,100,VERIFIED,2021-05-12 04:29:25+00:00,2022-02-22,S6QFtaJkO692FXM8Vjj2kw==,gs://gtfs-data-test/payments-processed/authori...,mst,2021-06-18 02:24:00,2,2,2
4,mst,9335c6ca-fd5a-41ac-9b35-c6d97eb2b14c,CS,CARD_CHECK,0.0,840,6207937648856229104091,100,VERIFIED,2021-05-12 04:29:25+00:00,2022-02-22,S6QFtaJkO692FXM8Vjj2kw==,gs://gtfs-data-test/payments-processed/authori...,mst,2021-06-18 02:24:00,2,2,1
5,mst,4a9e5e03-8174-46f7-a727-2bc15c79375b,CS,CARD_CHECK,0.0,840,6208373628036277803280,100,VERIFIED,2021-05-12 16:36:03+00:00,2022-02-22,LPTi4yxv7iA3FQIlVLmYvA==,gs://gtfs-data-test/payments-processed/authori...,mst,2021-06-18 02:24:00,2,2,1
6,mst,4a9e5e03-8174-46f7-a727-2bc15c79375b,CS,CARD_CHECK,0.0,840,6208373628036277803280,100,VERIFIED,2021-05-12 16:36:03+00:00,2022-02-22,LPTi4yxv7iA3FQIlVLmYvA==,gs://gtfs-data-test/payments-processed/authori...,mst,2021-06-18 02:24:00,2,2,2
7,mst,8c4b2b3c-1137-4410-9d95-363cb7e03c76,CS,CARD_CHECK,0.0,840,6208396169716260904066,100,VERIFIED,2021-05-12 17:13:37+00:00,2022-02-22,OIw44OdpQI55TQkksdzYpg==,gs://gtfs-data-test/payments-processed/authori...,mst,2021-06-18 02:24:00,2,2,1
8,mst,8c4b2b3c-1137-4410-9d95-363cb7e03c76,CS,CARD_CHECK,0.0,840,6208396169716260904066,100,VERIFIED,2021-05-12 17:13:37+00:00,2022-02-22,OIw44OdpQI55TQkksdzYpg==,gs://gtfs-data-test/payments-processed/authori...,mst,2021-06-18 02:24:00,2,2,2
9,mst,9f133f34-be3a-4864-bbf2-1ee9d788211d,CS,CARD_CHECK,0.0,840,6208414965046636204080,100,VERIFIED,2021-05-12 17:44:57+00:00,2022-02-22,35FbjXzx+CamNhL+tHrF3A==,gs://gtfs-data-test/payments-processed/authori...,mst,2021-06-18 02:24:00,1,1,1


### Findings

We probably want to join with the `micropayments` table on the `aggregation_id`. According to the LittlePay API docs document:

> The aggregation that is being authorised. A single aggregation can have multiple
> authorisations. The authorisations that are submitted depend on the scheme and the
> result of the initial authorisation (if declined, debt recovery authorisations may
> be performed).

It looks like there are duplicates (even just in the table above), so **we'll have to deduplicate on the hash**. No problem.

In [5]:
%%sql

# Number of unique records
select count(*)
from payments.stg_enriched_authorisations
where calitp_dupe_number = 1

,f0_
0,38490


### Next Step

We may want to ensure that the `aggregation_id` values are sensible with respect to the micropayments table.

In [6]:
%%sql

# Authorisations with no corresponding micropayment when joined by `aggregation_id`
select auths.*
from payments.stg_enriched_authorisations auths
full outer join payments.stg_cleaned_micropayments micropayments using (aggregation_id)
# where auths.aggregation_id is null
where (micropayments.aggregation_id is null and auths.calitp_dupe_number = 1)
limit 10

,participant_id,aggregation_id,acquirer_id,request_type,transaction_amount,currency_code,retrieval_reference_number,response_code,status,authorisation_date_time_utc,calitp_extracted_at,calitp_hash,calitp_file_name,calitp_export_account,calitp_export_datetime,calitp_n_dupes,calitp_n_dupe_ids,calitp_dupe_number
0,mst,31b6beb3-7fa0-42b4-96c6-a3bca2c92362,CS,CARD_CHECK,0.0,840,6454028847086012804013,100,VERIFIED,2022-02-21 00:21:25+00:00,2022-02-22,DvyCvHkKRlmjq/YKDeVsmQ==,gs://gtfs-data-test/payments-processed/authori...,mst,2022-02-22 05:01:00,1,1,1
1,mst,0c81ef4b-c2d3-4ccb-bbee-3cf5e8db9e3d,CS,DEBT_RECOVERY_AUTHCHECK,1.5,840,6213680524806948104275,208,DECLINED,2021-05-18 20:00:53+00:00,2022-02-22,tr12cBg+xMwPm4vWEthYUg==,gs://gtfs-data-test/payments-processed/authori...,mst,2021-06-18 02:24:00,2,2,1
2,mst,a3cd66c0-1827-48bf-a5d3-74349e7da4dd,CS,DEBT_RECOVERY_AUTHCHECK,7.0,840,6210765912686818104022,204,DECLINED,2021-05-15 11:03:11+00:00,2022-02-22,iV3dJwuoMj33hIHlktEn1w==,gs://gtfs-data-test/payments-processed/authori...,mst,2021-06-18 02:24:00,1,1,1
3,mst,44948539-d304-442e-bdb6-1efbb05fffc5,CS,DEBT_RECOVERY_AUTHCHECK,11.0,840,6207848509886821104020,204,DECLINED,2021-05-12 02:00:51+00:00,2022-02-22,ZIguuP3Zgafaviv8N6srXw==,gs://gtfs-data-test/payments-processed/authori...,mst,2021-06-18 02:24:00,13,13,1


Is this because of the status? Are only successfull authorisations linked with micropayments?

In [7]:
%%sql

select status, response_code, count(*)
from payments.stg_enriched_authorisations
group by 1, 2
order by 2, 1

,status,response_code,f0_
0,None,None,4
1,UNAVAILABLE,None,10
2,AUTHORISED,100,35286
3,VERIFIED,100,8514
4,INVALID,105,22
5,INVALID,150,3333
6,DECLINED,202,8
7,DECLINED,203,553
8,DECLINED,204,2775
9,LOST,205,32


### Findings

When joining auths with micropayments by `aggregation_id`, there are a few (4) authorisations records that have no corresponding micropayment. Why is that? It doesn't apper to have something to do with the response type; there are plenty of authorisations with non-`AUTHORISED` statuses that have associated micropayments.

### Next steps
* Ask Littlepay about this result
* Check on micropayments without corresponding authorisations

In [8]:
%%sql

# Micropayments with no corresponding authorisation when joined by `aggregation_id`
select micropayments.*
from payments.stg_enriched_authorisations auths
full outer join payments.stg_cleaned_micropayments micropayments using (aggregation_id)
where auths.aggregation_id is null
limit 10

,micropayment_id,aggregation_id,participant_id,customer_id,funding_source_vault_id,transaction_time,payment_liability,charge_amount,nominal_amount,currency_code,type,charge_type,calitp_extracted_at,calitp_hash,calitp_export_account,calitp_export_datetime
0,60294292-9aad-4aae-a36e-682d92e26f8e,d4c28b22-5212-4895-9ceb-55453754dcb0,clean-air-express,64e9f198-68cc-4b6b-9963-71672984200f,2632fe05-82e4-4d0f-99d9-022da09e5b43,2022-01-25 23:33:25+00:00,OPERATOR,0.00,6.00,840,DEBIT,flat_fare,2022-02-22,/Z8MTxIkY+NaFy9PnVg5Mg==,cal-itp,2022-01-26 05:43:00
1,215de51a-2e0b-4b3f-ac9f-a02357242b1c,3b2fc351-9080-4418-ae3b-35cb37963360,mst,3f9e3e46-b33f-4962-aaad-21fab46e2769,bdb897cc-1204-4d19-a6db-3f15e5497b6d,2021-07-28 14:03:39+00:00,OPERATOR,0.00,2.50,840,DEBIT,complete_variable_fare,2022-02-22,yvtbnOLHPUmsMLC04f/Klg==,mst,2021-07-30 05:33:00
2,27fde697-0da7-4d6b-8b81-fbb1638f6d31,3b2fc351-9080-4418-ae3b-35cb37963360,mst,3f9e3e46-b33f-4962-aaad-21fab46e2769,bdb897cc-1204-4d19-a6db-3f15e5497b6d,2021-07-29 00:48:20+00:00,OPERATOR,0.00,2.50,840,DEBIT,complete_variable_fare,2022-02-22,Vgfcy8b3p5FfSBKqJ772Xw==,mst,2021-07-30 05:33:00
3,3f0cf3e5-f370-41cc-b0d8-888e079e8823,3b2fc351-9080-4418-ae3b-35cb37963360,mst,3f9e3e46-b33f-4962-aaad-21fab46e2769,bdb897cc-1204-4d19-a6db-3f15e5497b6d,2021-07-28 14:19:04+00:00,OPERATOR,0.00,1.50,840,DEBIT,complete_variable_fare,2022-02-22,/kPqddrT2loH9/waW+GjLg==,mst,2021-07-30 05:33:00
4,f9ad535f-80b1-496a-ae7b-614a1da8ac48,fc4ce5cf-ffd2-4a9f-afe0-8adfa57d612b,mst,26b91a6d-829f-4271-b3cb-202df4119648,133c17ea-d5e0-4cd3-b3cf-8f66edbf7bf6,2022-02-01 23:51:41+00:00,OPERATOR,1.50,1.50,840,DEBIT,complete_variable_fare,2022-02-22,oY48gUl4mbvYyenuJPjJfQ==,mst,2022-02-03 05:37:00
5,167926bc-4f14-464d-868f-41ded7bcd056,d183e6a5-73f1-4cfc-b27e-9dc3ab6037d1,sacrt,25b47e24-c0eb-4aa2-9d81-bc3d54faa73f,65045422-d758-4360-9251-d3b934d69810,2021-08-31 17:52:38+00:00,OPERATOR,1.00,1.00,840,DEBIT,flat_fare,2022-02-22,YADzBU2VrPe3slItvwFMYA==,cal-itp,2021-10-18 23:14:00
6,65a87aa7-c229-4ea1-b8bd-790f6544fdd0,abb905f8-78f7-4674-b729-f1815bd19256,sbmtd,d5709812-5077-448c-abc1-dba016e7031e,1e560a56-faa4-4018-baba-03bc60c11855,2021-08-25 22:43:45+00:00,OPERATOR,1.75,1.75,840,DEBIT,flat_fare,2022-02-22,5/h4Sdrms01sNMBda1WwNQ==,sbmtd,2021-10-18 23:16:00
7,d94c82e2-1ae1-4da4-a410-9a9c273ac866,a973656f-06e7-4e34-a323-6dff5500cf01,sbmtd,c1853fa7-e9ff-4b65-a180-67053df205f1,2f22592a-b2a1-479d-b010-27dab665bfa0,2021-09-10 18:12:45+00:00,OPERATOR,1.75,1.75,840,DEBIT,flat_fare,2022-02-22,wDUrz1rkZyundb1ZHxnSbQ==,sbmtd,2021-10-18 23:16:00
8,f9a18f0c-fc5c-4d5b-8e28-a93678b8539b,b48f994f-ffb2-440e-9337-799b698b1e29,sbmtd,0242996f-0e90-4b87-ae48-d670f023aa3d,87ccfc7b-c79c-442b-ab69-c13688df7103,2021-08-11 17:02:22+00:00,OPERATOR,1.75,1.75,840,DEBIT,flat_fare,2022-02-22,Q577fd486nLs+CZEf9z33Q==,sbmtd,2021-10-18 23:16:00
9,601da923-3442-4136-83ba-c816749761b9,6c988377-727a-4c9b-92d4-83ee751eed29,sbmtd,895ad725-5668-4d30-bfbb-0e38bf0577b4,90339c71-90f4-4c8c-91e6-1dd0198669a5,2021-08-20 01:02:14+00:00,OPERATOR,1.75,1.75,840,DEBIT,flat_fare,2022-02-22,uBxV9C1kBa7JtsjNvn45Pg==,sbmtd,2021-10-18 23:16:00


In [9]:
%%sql

# How many micropayments with no corresponding authorisation when joined by `aggregation_id`
select count(*)
from payments.stg_enriched_authorisations auths
full outer join payments.stg_cleaned_micropayments micropayments using (aggregation_id)
where auths.aggregation_id is null

,f0_
0,3447


### Findings

There are also many micropayments with no corresponding authorisations.

Nevertheless, it looks like the bulk of authorizations have micropayments and vice versa. Perhaps that's good enough to start.
